In [6]:
%pip install emoji

  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
import emoji
import unicodedata

In [2]:
chunk1 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk1.csv')
chunk2 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk2.csv')
chunk3 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk3.csv')
chunk4 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk4.csv')
chunk5 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk5.csv')
chunk6 = pd.read_csv('raw_data/train-balanced-sarcasm_chunk6.csv')

df = pd.concat([chunk1, chunk2, chunk3, chunk4, chunk5, chunk6], ignore_index=True)


Filter only sarcasm comment and long text

In [3]:
df = df[df['label'] == 1]
df = df[df['comment'].str.len() > 30]

Filter game comments

In [4]:
game_subreddits = [
    'gaming', 'pcgaming', 'games', 'leagueoflegends', 'Overwatch',
    'GlobalOffensive', 'FortNiteBR', 'PS4', 'xboxone', 'wow', 'nintendo', 'Minecraft'
]

df_game = df[df['subreddit'].isin(game_subreddits)]

Select only ```comment``` column

In [5]:
df_game = df_game[['comment']]

Handle duplicates

In [6]:
df_game.duplicated().sum()

27

In [7]:
df_game = df_game.drop_duplicates()

Check missing

In [8]:
df_game.isna().sum()

comment    0
dtype: int64

Normalize text

In [9]:
def normalize_text(text):
    text = text.lower()  # lowercase

    # remove 'early access review' at the beginning
    text = re.sub(r"^(early access review[\s:\-–—)]*)", "", text, flags=re.IGNORECASE)

    # remove URLs, mentions, hashtags
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)

    # remove numbers
    #text = re.sub(r"\d+", "", text)

    # remove emojis and icons
    text = emoji.replace_emoji(text, replace='')

    # Remove spam patterns: no more than 3 consecutive identical characters
    # e.g., "goooood" → "good", "aaaaawesome" → "awesome"
    text = re.sub(r'(.)\1{3,}', r'\1\1\1', text)  # Keep up to 3 repeats
    
    # remove non-printable or control characters + double quotes
    text = ''.join(
        c for c in text 
        if unicodedata.category(c)[0] != 'C' and c.isprintable() and c != '"'
    )

    # normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # remove duplicate consecutive words
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

    # keep only ASCII characters (English text only)
    text = text.encode("ascii", "ignore").decode()

    return text.strip()

# Apply to the DataFrame
df_game['comment'] = df_game['comment'].apply(normalize_text)

In [10]:
df_game
df_game.to_csv('sarcasm_raw.csv')

In [11]:
from openai import OpenAI
import os
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv
import csv


Label comments

In [ ]:
load_dotenv()


df = pd.read_csv("sarcasm_raw.csv")
results = []

if os.path.exists("sarcasm_labeled.csv"):
    dups = pd.read_csv("sarcasm_labeled.csv")
    for _, row in dups.iterrows():
        results.append(row)
else:
    with open("sarcasm_labeled.csv", "w",newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['comment', 'user_suggestion'])

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("api_key"))


# Process unprocessed rows in batches
for idx, row in df.iterrows():
    if idx < len(results):
        continue

    prompt = (
        'Your role is do sentiment analysis on sarcastic comments from social medias on topic relating to video games.\n'

        'If the comment is positive, return the number "1"\n'

        'If the comment is negative, return the number "0"\n'

        'Return only the sentiment result and nothing else.\n'

        'Here is the comment:\n'

        f'{row["comment"]}'
    )

    completion = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "user", "content": prompt}]
    )


    #Extract response and parse JSON
    try:
        response_data = int(completion.choices[0].message.content)
        print(idx,response_data)

        with open("sarcasm_labeled.csv", "a",newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([row['comment'], response_data])



    except json.JSONDecodeError:
        print("Error parsing response. Skipping.")

print("Processing complete! Only new rows were added.")
